In [1]:
import os
import numpy as np
import pandas as pd
import iris
import iris.coord_categorisation
import iris.analysis
from iris.cube import CubeList
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from matplotlib.colors import TwoSlopeNorm
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import geopandas as gpd
import cf_units
import cartopy.crs as ccrs
import basic_info

# Specify shapefile to work with
shapefile = gpd.read_file(basic_info.shapefile_outline)
location_name = basic_info.location_name

# Creating a folder for outputs
output_folder = f'{basic_info.directory_filepath}/outputs'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    with open('basic_info.py', 'a') as f:
            f.write(f'\noutput_folder = "{output_folder}"')

In [8]:
# Useful functions

def add_season_year_dim(cube, seasons=None):
    '''A function that returns a cube with the season_year attached as a
    dimension

    Input args:
    ----------
    cube  -- the cube to add the season_year to
    '''
    try:
        cube.remove_coord('season_year')
    except iris.exceptions.CoordinateNotFoundError:
        pass
    if seasons:
        iris.coord_categorisation.add_season_year(cube, 'time', name='season_year',
                                        seasons=seasons)
    else:
        iris.coord_categorisation.add_season_year(cube, 'time', name='season_year',
                                        seasons=('djf','mam','jja','son')) # Define if different seasons are needed
    return cube

def add_season_dim(cube):
    '''A function that returns a cube with the season attached as a
    dimension

    Input args:
    ----------
    cube  -- the cube to add the season to
    '''
    try:
        cube.remove_coord('season')
    except iris.exceptions.CoordinateNotFoundError:
        pass
    iris.coord_categorisation.add_season(cube, 'time', name='season',
                                         seasons=('djf','mam','jja','son'))
    return cube


def add_day_of_month_dim(cube):
    '''A function that returns a cube with the day of the month attached as a
    dimension

    Input args:
    ----------
    cube  -- the cube to add the day of the month to
    '''
    try:
        cube.remove_coord('day_of_month')
    except iris.exceptions.CoordinateNotFoundError:
        pass
    iris.coord_categorisation.add_day_of_month(cube, 'time', name='day_of_month')
    return cube

def add_day_of_year_dim(cube):
    '''A function that returns a cube with the day of the year attached as a
    dimension

    Input args:
    ----------
    cube  -- the cube to add the day of the year to
    '''
    try:
        cube.remove_coord('day_of_year')
    except iris.exceptions.CoordinateNotFoundError:
        pass
    iris.coord_categorisation.add_day_of_year(cube, 'time', name='day_of_year')

def add_month_dim(cube):
    '''A function that returns a cube with the month attached as a
    dimension

    Input args:
    ----------
    cube  -- the cube to add the month to
    '''
    try:
        cube.remove_coord('month')
    except iris.exceptions.CoordinateNotFoundError:
        pass
    iris.coord_categorisation.add_month(cube, 'time', name='month')
    return cube

def add_hour_dim(cube):
    '''A function that returns a cube with the hour attached as a
    dimension

    Input args:
    ----------
    cube  -- the cube to add the hour to
    '''
    try:
        cube.remove_coord('hour')
    except iris.exceptions.CoordinateNotFoundError:
        pass
    iris.coord_categorisation.add_hour(cube, 'time', name='hour')
    return cube

def hourly_to_daily_max(cube):
    # Add day, month, and season year dimensions
    add_day_month_season_year_dims(cube)
    
    # Aggregate by day of month, month, and season year
    daily_max = cube.aggregated_by(['day_of_month', 'month', 'season_year'], iris.analysis.MAX)
    
    return daily_max

In [9]:
def load_my_files():
    heat_index_cube = iris.load_cube(basic_info.heat_index_cube)
    print(f'Loaded Heat Index cube as heat_index_cube')

    print('Adding useful information to the cube files')
    heat_index_cube.convert_units('Kelvin')
    add_day_of_year_dim(heat_index_cube)
    add_season_year_dim(heat_index_cube)
    add_month_dim(heat_index_cube)
    
    heat_index_cube.long_name = 'Lu and Romps Heat Index'

    tas_cube = iris.load_cube(basic_info.tas_filepath)
    print('Loaded tas cube')
    rh_cube = iris.load_cube(basic_info.rh_filepath)
    print('Loaded rh cube')

    tas_cube.convert_units('Kelvin')

    add_season_year_dim(tas_cube)
    add_season_year_dim(rh_cube)
    add_month_dim(tas_cube)
    add_month_dim(rh_cube)
    add_hour_dim(tas_cube)
    add_hour_dim(rh_cube)
    
    return heat_index_cube, tas_cube, rh_cube
    

## Creation of variables

In [10]:
def collapse_air_temperature_for_time_series_of_average_season_year_max_cell_values():
    global tas_time_series_cube
    tas_time_series_cube = tas_cube.aggregated_by('season_year', iris.analysis.MAX)
    tas_time_series_cube = tas_time_series_cube.collapsed(['latitude','longitude'], iris.analysis.MEAN)
    return tas_time_series_cube

def collapse_air_temperature_for_maps_of_average_season_year_max_cell_values():
    global tas_map_cube
    tas_map_cube = tas_cube.aggregated_by('season_year', iris.analysis.MAX)
    tas_map_cube = tas_cube.collapsed('time', iris.analysis.MEAN)
    return tas_map_cube

## Basic Plots

In [11]:
def plot_my_region():
    fig, ax = plt.subplots()
    shapefile.plot(ax=ax, facecolor='antiquewhite', edgecolor='black')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title(f'{location_name}')
    plt.show()

In [ ]:
def calculate_statistics_max_yearly(cube):
    min_val = cube.aggregated_by('season_year', iris.analysis.MIN)
    min_val = min_val.collapsed(['latitude', 'longitude'], iris.analysis.MAX).data

    lower_quartile = cube.aggregated_by('season_year', iris.analysis.PERCENTILE, percent=25)
    lower_quartile = lower_quartile.collapsed(['latitude', 'longitude'], iris.analysis.MAX).data

    mean_val = cube.aggregated_by('season_year', iris.analysis.MEAN)
    mean_val = mean_val.collapsed(['latitude', 'longitude'], iris.analysis.MAX).data

    median_val = cube.aggregated_by('season_year', iris.analysis.MEDIAN)
    median_val = median_val.collapsed(['latitude', 'longitude'], iris.analysis.MAX).data

    upper_quartile = cube.aggregated_by('season_year', iris.analysis.PERCENTILE, percent=75)
    upper_quartile = upper_quartile.collapsed(['latitude', 'longitude'], iris.analysis.MAX).data

    max_val = cube.aggregated_by('season_year', iris.analysis.MAX)
    max_val = max_val.collapsed(['latitude', 'longitude'], iris.analysis.MAX).data

    return min_val, lower_quartile, mean_val, median_val, upper_quartile, max_val

def calculate_statistics_mean_yearly(cube):
    min_val = cube.aggregated_by('season_year', iris.analysis.MIN)
    min_val = min_val.collapsed(['latitude', 'longitude'], iris.analysis.MEAN).data

    lower_quartile = cube.aggregated_by('season_year', iris.analysis.PERCENTILE, percent=25)
    lower_quartile = lower_quartile.collapsed(['latitude', 'longitude'], iris.analysis.MEAN).data

    mean_val = cube.aggregated_by('season_year', iris.analysis.MEAN)
    mean_val = mean_val.collapsed(['latitude', 'longitude'], iris.analysis.MEAN).data

    median_val = cube.aggregated_by('season_year', iris.analysis.MEDIAN)
    median_val = median_val.collapsed(['latitude', 'longitude'], iris.analysis.MEAN).data

    upper_quartile = cube.aggregated_by('season_year', iris.analysis.PERCENTILE, percent=75)
    upper_quartile = upper_quartile.collapsed(['latitude', 'longitude'], iris.analysis.MEAN).data

    max_val = cube.aggregated_by('season_year', iris.analysis.MAX)
    max_val = max_val.collapsed(['latitude', 'longitude'], iris.analysis.MEAN).data

    return min_val, lower_quartile, mean_val, median_val, upper_quartile, max_val

def calculate_statistics_max_monthly(cube):
    min_val = cube.aggregated_by(['season_year', 'month'], iris.analysis.MIN)
    min_val = min_val.collapsed(['latitude', 'longitude'], iris.analysis.MAX).data
    
    lower_quartile = cube.aggregated_by(['season_year', 'month'], iris.analysis.PERCENTILE, percent=25)
    lower_quartile = lower_quartile.collapsed(['latitude', 'longitude'], iris.analysis.MAX).data
    
    mean_val = cube.aggregated_by(['season_year', 'month'], iris.analysis.MEAN)
    mean_val = mean_val.collapsed(['latitude', 'longitude'], iris.analysis.MAX).data
    
    median_val = cube.aggregated_by(['season_year', 'month'], iris.analysis.MEDIAN)
    median_val = median_val.collapsed(['latitude', 'longitude'], iris.analysis.MAX).data
    
    upper_quartile = cube.aggregated_by(['season_year', 'month'], iris.analysis.PERCENTILE, percent=75)
    upper_quartile = upper_quartile.collapsed(['latitude', 'longitude'], iris.analysis.MAX).data
    
    max_val = cube.aggregated_by(['season_year', 'month'], iris.analysis.MAX)
    max_val = max_val.collapsed(['latitude', 'longitude'], iris.analysis.MAX).data
    
    return min_val, lower_quartile, mean_val, median_val, upper_quartile, max_val

def calculate_statistics_mean_monthly(cube):
        min_val = cube.aggregated_by(['season_year', 'month'], iris.analysis.MIN)
        min_val = min_val.collapsed(['latitude', 'longitude'], iris.analysis.MEAN).data

        lower_quartile = cube.aggregated_by(['season_year', 'month'], iris.analysis.PERCENTILE, percent=25)
        lower_quartile = lower_quartile.collapsed(['latitude', 'longitude'], iris.analysis.MEAN).data

        mean_val = cube.aggregated_by(['season_year', 'month'], iris.analysis.MEAN)
        mean_val = mean_val.collapsed(['latitude', 'longitude'], iris.analysis.MEAN).data

        median_val = cube.aggregated_by(['season_year', 'month'], iris.analysis.MEDIAN)
        median_val = median_val.collapsed(['latitude', 'longitude'], iris.analysis.MEAN).data

        upper_quartile = cube.aggregated_by(['season_year', 'month'], iris.analysis.PERCENTILE, percent=75)
        upper_quartile = upper_quartile.collapsed(['latitude', 'longitude'], iris.analysis.MEAN).data

        max_val = cube.aggregated_by(['season_year', 'month'], iris.analysis.MAX)
        max_val = max_val.collapsed(['latitude', 'longitude'], iris.analysis.MEAN).data

        return min_val, lower_quartile, mean_val, median_val, upper_quartile, max_val

def calculate_statistics_max_tas_and_rh(cube):
        min_val = cube.aggregated_by('month', iris.analysis.MIN)
        min_val = min_val.collapsed(['latitude', 'longitude'], iris.analysis.MAX).data

        lower_quartile = cube.aggregated_by(['season_year', 'month'], iris.analysis.PERCENTILE, percent=25)
        lower_quartile = lower_quartile.collapsed(['latitude', 'longitude'], iris.analysis.MAX).data

        mean_val = cube.aggregated_by('month', iris.analysis.MEAN)
        mean_val = mean_val.collapsed(['latitude', 'longitude'], iris.analysis.MAX).data

        median_val = cube.aggregated_by('month', iris.analysis.MEDIAN)
        median_val = median_val.collapsed(['latitude', 'longitude'], iris.analysis.MAX).data

        upper_quartile = cube.aggregated_by('month', iris.analysis.PERCENTILE, percent=75)
        upper_quartile = upper_quartile.collapsed(['latitude', 'longitude'], iris.analysis.MAX).data

        max_val = cube.aggregated_by('month', iris.analysis.MAX)
        max_val = max_val.collapsed(['latitude', 'longitude'], iris.analysis.MAX).data

        return min_val, lower_quartile, mean_val, median_val, upper_quartile, max_val

def calculate_statistics_mean_tas_and_rh(cube):
        min_val = cube.aggregated_by('month', iris.analysis.MIN)
        min_val = min_val.collapsed(['latitude', 'longitude'], iris.analysis.MEAN).data

        lower_quartile = cube.aggregated_by('month', iris.analysis.PERCENTILE, percent=25)
        lower_quartile = lower_quartile.collapsed(['latitude', 'longitude'], iris.analysis.MEAN).data

        mean_val = cube.aggregated_by('month', iris.analysis.MEAN)
        mean_val = mean_val.collapsed(['latitude', 'longitude'], iris.analysis.MEAN).data

        median_val = cube.aggregated_by('month', iris.analysis.MEDIAN)
        median_val = median_val.collapsed(['latitude', 'longitude'], iris.analysis.MEAN).data

        upper_quartile = cube.aggregated_by('month', iris.analysis.PERCENTILE, percent=75)
        upper_quartile = upper_quartile.collapsed(['latitude', 'longitude'], iris.analysis.MEAN).data

        max_val = cube.aggregated_by('month', iris.analysis.MAX)
        max_val = max_val.collapsed(['latitude', 'longitude'], iris.analysis.MEAN).data

        return min_val, lower_quartile, mean_val, median_val, upper_quartile, max_val

In [12]:
def calculate_mean_tas_per_month():
    mean_tas_over_space = tas_cube.collapsed(['latitude', 'longitude'], iris.analysis.MEAN)
    mean_tas_per_month = mean_tas_over_space.aggregated_by('month', iris.analysis.MEAN)
    return mean_tas_per_month

def calculate_mean_rh_per_month():
    mean_rh_over_space = rh_cube.collapsed(['latitude', 'longitude'], iris.analysis.MEAN)
    mean_rh_per_month = mean_rh_over_space.aggregated_by('month', iris.analysis.MEAN)
    return mean_rh_per_month

def plot_monthly_average_tas_and_rh():
    fig, axes = plt.subplots(nrows=1, ncols=2)
    axes=axes.flatten()
    
    months = np.unique(tas_cube.coord('month').points)
    
    tas_min, tas_lower_quartile, tas_mean, tas_median, tas_upper_quartile, tas_max = calculate_statistics_mean_tas_and_rh(tas_cube)
    rh_min, rh_lower_quartile, rh_mean, rh_median, rh_upper_quartile, rh_max = calculate_statistics_mean_tas_and_rh(rh_cube)
    
    # Plot the average temperature per month
    
    axes[0].plot(months, tas_min, color='blue', label='Heat Min', linestyle ='-', marker='o')
    axes[0].plot(months, tas_lower_quartile, color='green', label='Heat 25th Percentile', linestyle ='-', marker='v')
    axes[0].plot(months, tas_mean, color='red', label='Heat Mean', linestyle ='-', marker='s')
    axes[0].plot(months, tas_median, color='purple', label='Heat Median', linestyle ='-', marker='D')
    axes[0].plot(months, tas_upper_quartile, color='orange', label='Heat 75th Percentile', linestyle ='-', marker='^')
    axes[0].plot(months, tas_max, color='black', label='Heat Max', linestyle ='-', marker='x')

    axes[0].set_xlabel('Month')
    axes[0].set_ylabel('Temperature (K)')

    axes[1].plot(months, rh_min, color='blue', label='Heat Min', linestyle ='-', marker='o')
    axes[1].plot(months, rh_lower_quartile, color='green', label='Heat 25th Percentile', linestyle ='-', marker='v')
    axes[1].plot(months, rh_mean, color='red', label='Heat Mean', linestyle ='-', marker='s')
    axes[1].plot(months, rh_median, color='purple', label='Heat Median', linestyle ='-', marker='D')
    axes[1].plot(months, rh_upper_quartile, color='orange', label='Heat 75th Percentile', linestyle ='-', marker='^')
    axes[1].plot(months, rh_max, color='black', label='Heat Max', linestyle ='-', marker='x')

    axes[1].set_xlabel('Month')
    axes[1].set_ylabel('Humidity (%)')

    handles, labels = axes[1].get_legend_handles_labels()
    labels = ['Min', '25th Percentile', 'Mean', 'Median', '75th Percentile', 'Max'] # Updating the labels so they are only shown once
    fig.legend(handles, labels, loc='lower center', ncol=len(labels)/2, bbox_to_anchor=(0.5, -0.1)) # While loc already sets the legend to the centre, bbox_to_anchor's second value moves it down so it's not on top of the title

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.suptitle(f'Spatial Mean Air Temperature and Humidity per Month ({start_year}-{end_year})')
    plt.savefig(f'{os.getcwd()}/outputs/average_air_temperature_and_humidity_per_month.png')
    plt.show()

In [13]:
# Check the mean average heat index per hour for the DJF season across space and time
def calculate_mean_tas_per_hour():
    mean_tas_per_hour = tas_cube.collapsed(['latitude', 'longitude'], iris.analysis.MEAN)
    mean_tas_per_hour = mean_tas_per_hour.aggregated_by('hour', iris.analysis.MEAN)
    return mean_tas_per_hour

def calculate_mean_rh_per_hour():
    mean_rh_per_hour = rh_cube.collapsed(['latitude', 'longitude'], iris.analysis.MEAN)
    mean_rh_per_hour = mean_rh_per_hour.aggregated_by('hour', iris.analysis.MEAN)
    return mean_rh_per_hour

def plot_hourly_average_tas_and_rh():

    # Create a dataframe of the mean temperature and humidity per hour to plot
    mean_tas_per_hour = calculate_mean_tas_per_hour()
    mean_rh_per_hour = calculate_mean_rh_per_hour()

    mean_values_per_hour_dict = {'hour': mean_tas_per_hour.coord('hour').points,
                                'mean_tas': mean_tas_per_hour.data,
                                'mean_rh': mean_rh_per_hour.data}

    mean_values_per_hour_df = pd.DataFrame(mean_values_per_hour_dict)
    mean_values_per_hour_df = mean_values_per_hour_df.sort_values(by='hour').reset_index(drop=True)
    mean_values_per_hour_df

    fig = plt.figure(figsize=[10, 5])

    ## Plot of the average temperature per hour

    fig, ax = plt.subplots()
    plt.plot(mean_values_per_hour_df['hour'], mean_values_per_hour_df['mean_tas'], label='tas', color='red', marker='o')

    ax.set_xlabel('Hour')
    ax.set_ylabel('Temperature (˚C)')
    ax.set_xticks(np.arange(0, 24, 1))
    ax.set_xlim(0, 23)
    plt.title(f'Average Temperature per Hour ({start_year}-{end_year})')
    plt.savefig(f'{output_folder}/{location_name}_average_temperature_per_hour.png')
    plt.show()

    # Plots
    ## Plot of the average relative humidity per hour
    fig = plt.figure(figsize=[10, 5])

    fig, ax = plt.subplots()
    plt.plot(mean_values_per_hour_df['hour'], mean_values_per_hour_df['mean_rh'], label='rh', color='blue', marker='o')

    ax.set_xlabel('Hour')
    ax.set_ylabel('Relative Humidity (%)')
    ax.set_xticks(np.arange(0, 24, 1))
    ax.set_xlim(0, 23)
    plt.title(f'Average Relative Humidity per Hour ({start_year}-{end_year})')
    plt.savefig(f'{output_folder}/{location_name}_average_relative_humidity_per_hour.png')
    plt.show()


    ## Combined plot per hour
    fig, ax_temp = plt.subplots()
    ax_temp.plot(mean_values_per_hour_df['hour'], mean_values_per_hour_df['mean_tas'], label='Temperature', color='red', marker='o')

    ax_temp.set_xticks(np.arange(0, 24, 1))
    ax_temp.set_xlim(0, 23)

    ax_temp.set_xlabel('Hour')
    ### Set labels for primary axis
    ax_temp.set_ylabel('Temperature (˚C)', color='red')
    ax_temp.tick_params(axis='y', labelcolor='red')

    ### Create secondary axis for humidity
    ax_hum = ax_temp.twinx()
    ax_hum.plot(mean_values_per_hour_df['hour'], mean_values_per_hour_df['mean_rh'],label='Humidity', color='blue', marker='o')

    ### Set labels for secondary axis
    ax_hum.set_ylabel('Humidity (%)', color='blue')
    ax_hum.tick_params(axis='y', labelcolor='blue')

    ### Title
    plt.title(f'Average Temperature and Humidity per Hour ({start_year}-{end_year})')

    ### Save and show plot
    plt.savefig(f'{output_folder}/{location_name}_average_temperature_and_humidity_per_hour.png')
    plt.show()

    return mean_tas_per_hour, mean_rh_per_hour

In [14]:
# Instruction: adjust the below to show variables you are interested in.
def create_season_year_aggregates():
    global tas_cube_yearly_mean, rh_cube_yearly_mean, tas_cube_yearly_min, rh_cube_yearly_min, tas_cube_yearly_max, rh_cube_yearly_max
    
    tas_cube_yearly_mean = tas_cube.aggregated_by('season_year', iris.analysis.MEAN)
    tas_cube_yearly_mean = tas_cube_yearly_mean.collapsed(['latitude', 'longitude'], iris.analysis.MEAN)
    rh_cube_yearly_mean = rh_cube.aggregated_by('season_year', iris.analysis.MEAN)
    rh_cube_yearly_mean = rh_cube_yearly_mean.collapsed(['latitude', 'longitude'], iris.analysis.MEAN)

    # Min cubes
    tas_cube_yearly_min = tas_cube.aggregated_by('season_year', iris.analysis.MIN)
    tas_cube_yearly_min = tas_cube_yearly_min.collapsed(['latitude', 'longitude'], iris.analysis.MIN)
    rh_cube_yearly_min = rh_cube.aggregated_by('season_year', iris.analysis.MIN)
    rh_cube_yearly_min = rh_cube_yearly_min.collapsed(['latitude', 'longitude'], iris.analysis.MIN)

    # Max cubes
    tas_cube_yearly_max = tas_cube.aggregated_by('season_year', iris.analysis.MAX)
    tas_cube_yearly_max = tas_cube_yearly_max.collapsed(['latitude', 'longitude'], iris.analysis.MAX)
    rh_cube_yearly_max = rh_cube.aggregated_by('season_year', iris.analysis.MAX)
    rh_cube_yearly_max = rh_cube_yearly_max.collapsed(['latitude', 'longitude'], iris.analysis.MAX)

def create_overall_aggregates():
    global tas_cube_overall_mean, rh_cube_overall_mean, tas_cube_overall_min, rh_cube_overall_min, tas_cube_overall_max, rh_cube_overall_max
# Calculating the OVERALL () mean, min, and max for each variable
    # Mean cubes
    tas_cube_overall_mean = tas_cube.aggregated_by('season_year', iris.analysis.MEAN)
    tas_cube_overall_mean = tas_cube_overall_mean.collapsed('time', iris.analysis.MEAN)
    rh_cube_overall_mean = rh_cube.aggregated_by('season_year', iris.analysis.MEAN)
    rh_cube_overall_mean = rh_cube_overall_mean.collapsed('time', iris.analysis.MEAN)

    # Min cubes
    tas_cube_overall_min = tas_cube.aggregated_by('season_year', iris.analysis.MIN)
    tas_cube_overall_min = tas_cube_overall_min.collapsed('time', iris.analysis.MIN)
    rh_cube_overall_min = rh_cube.aggregated_by('season_year', iris.analysis.MIN)
    rh_cube_overall_min = rh_cube_overall_min.collapsed('time', iris.analysis.MIN)

    # Max cubes
    tas_cube_overall_max = tas_cube.aggregated_by('season_year', iris.analysis.MAX)
    tas_cube_overall_max = tas_cube_overall_max.collapsed('time', iris.analysis.MAX)
    rh_cube_overall_max = rh_cube.aggregated_by('season_year', iris.analysis.MAX)
    rh_cube_overall_max = rh_cube_overall_max.collapsed('time', iris.analysis.MAX)

def plot_min_mean_max_analysis_maps():
    fig = plt.figure(figsize=(17, 19))
    fig.subplots_adjust(left=0.2)  # Adjust the left padding to increase space for y-labels and titles
    subfigs = fig.subfigures(3, 1)
    
    subfigs[0].suptitle('Min', x=0.05, y=0.6, ha='left', va='center', fontsize=14, rotation=90)
    subfigs[1].suptitle('Mean', x=0.05, y=0.6, ha='left', va='center', fontsize=14, rotation=90)
    subfigs[2].suptitle('Max', x=0.05, y=0.6, ha='left', va='center', fontsize=14, rotation=90)

    axstop = subfigs[0].subplots(1, 2, sharex=True, sharey=True)
    axsmid = subfigs[1].subplots(1, 2, sharex=True, sharey=True)
    axsbott = subfigs[2].subplots(1, 2, sharex=True, sharey=True)

    longitude = tas_cube_overall_mean.coord('longitude').points
    latitude = tas_cube_overall_mean.coord('latitude').points
    lon, lat = np.meshgrid(longitude, latitude)  # Create a meshgrid for plotting

    # Plot 1: Yearly summer min temperature at surface
    mesh1 = axstop[0].pcolormesh(lon, lat, tas_cube_overall_min.data, cmap='Reds')
    cbar1 = fig.colorbar(mesh1, ax=axstop[0], orientation='horizontal')
    cbar1.set_ticks(np.linspace(tas_cube_overall_min.data.min(), tas_cube_overall_min.data.max(), 5))
    cbar1.ax.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))  # Format ticks to 1 decimal place
    axstop[0].set_xlabel('Longitude')
    axstop[0].set_ylabel('Latitude')

    shapefile.plot(ax=axstop[0], color='none', edgecolor='black', linewidth=1)  # Overlay the shape file on the first plot
    axstop[0].set_title(f'Temperature at Surface \n', fontsize=14)

    # Plot 2: Yearly summer mean humidity
    mesh2 = axstop[1].pcolormesh(lon, lat, rh_cube_overall_min.data, cmap='Blues')
    cbar2 = fig.colorbar(mesh2, ax=axstop[1], orientation='horizontal')
    cbar2.set_ticks(np.linspace(rh_cube_overall_min.data.min(), rh_cube_overall_min.data.max(), 5))
    cbar2.ax.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))  # Format ticks to 1 decimal place
    axstop[1].set_xlabel('Longitude')
    axstop[1].set_ylabel('Latitude')

    shapefile.plot(ax=axstop[1], color='none', edgecolor='black', linewidth=1)  # Overlay the shape file on the first plot
    axstop[1].set_title(f'Humidity (%) \n', fontsize=14)

    # Plot 3: Yearly summer mean temperature at surface
    mesh3 = axsmid[0].pcolormesh(lon, lat, tas_cube_overall_mean.data, cmap='Reds')
    cbar3 = fig.colorbar(mesh3, ax=axsmid[0], orientation='horizontal')
    cbar3.set_ticks(np.linspace(tas_cube_overall_mean.data.min(), tas_cube_overall_mean.data.max(), 5))
    cbar3.ax.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))  # Format ticks to 1 decimal place
    axsmid[0].set_xlabel('Longitude')
    axsmid[0].set_ylabel('Latitude')

    shapefile.plot(ax=axsmid[0], color='none', edgecolor='black', linewidth=1)  # Overlay the shape file on the first plot

    # Plot 4: Yearly summer mean humidity
    mesh4 = axsmid[1].pcolormesh(lon, lat, rh_cube_overall_mean.data, cmap='Blues')
    cbar4 = fig.colorbar(mesh4, ax=axsmid[1], orientation='horizontal')
    cbar4.set_ticks(np.linspace(rh_cube_overall_mean.data.min(), rh_cube_overall_mean.data.max(), 5))
    cbar4.ax.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))  # Format ticks to 1 decimal place
    axsmid[1].set_xlabel('Longitude')
    axsmid[1].set_ylabel('Latitude')

    shapefile.plot(ax=axsmid[1], color='none', edgecolor='black', linewidth=1)  # Overlay the shape file on the first plot

    # Plot 5: Yearly summer max temperature at surface
    mesh5 = axsbott[0].pcolormesh(lon, lat, tas_cube_overall_max.data, cmap='Reds')
    cbar5 = fig.colorbar(mesh5, ax=axsbott[0], orientation='horizontal')
    cbar5.set_ticks(np.linspace(tas_cube_overall_max.data.min(), tas_cube_overall_max.data.max(), 5))
    cbar5.ax.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))  # Format ticks to 1 decimal place
    axsbott[0].set_xlabel('Longitude')
    axsbott[0].set_ylabel('Latitude')
    shapefile.plot(ax=axsbott[0], color='none', edgecolor='black', linewidth=1)  # Overlay the shape file on the first plot

    # Plot 6: Yearly summer max humidity
    mesh6 = axsbott[1].pcolormesh(lon, lat, rh_cube_overall_max.data, cmap='Blues')
    cbar6 = fig.colorbar(mesh6, ax=axsbott[1], orientation='horizontal')
    cbar6.set_ticks(np.linspace(rh_cube_overall_max.data.min(), rh_cube_overall_max.data.max(), 5))
    cbar6.ax.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))  # Format ticks to 1 decimal place
    axsbott[1].set_xlabel('Longitude')
    axsbott[1].set_ylabel('Latitude')
    shapefile.plot(ax=axsbott[1], color='none', edgecolor='black', linewidth=1)  # Overlay the shape file on the first plot
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.savefig(f'{output_folder}/{location_name}_analysis_maps.png')
    plt.show()

In [ ]:
def temporally_plot_monthly_mean_values_from_Lu_and_Romps_against_air_temperature():
    fig, ax = plt.subplots(figsize=(12, 8))

    # Calculate min, lower quartile, mean, median, upper quartile, and max for the heat_index_cube and tas_cube
    heat_min, heat_lower_quartile, heat_mean, heat_median, heat_upper_quartile, heat_max = calculate_statistics_max_yearly(heat_index_cube)
    tas_min, tas_lower_quartile, tas_mean, tas_median, tas_upper_quartile, tas_max = calculate_statistics_max_yearly(tas_cube)  


    # Creating "aggregated_cube" so I can use it across all time plots to draw coordinates from without having to specify a new file each time (gets confusing otherwise)
    aggregated_cube = heat_index_cube.aggregated_by('season_year', iris.analysis.MIN)
    aggregated_cube = aggregated_cube.collapsed(['latitude', 'longitude'], iris.analysis.MEAN)

    time_points = aggregated_cube.coord('time').points
    time_unit = aggregated_cube.coord('time').units

    # Convert numerical time points to actually readable dates
    readable_dates = cf_units.num2date(time_points, time_unit.origin, time_unit.calendar)
    formatted_dates = [date.strftime('%Y-%m') for date in readable_dates]

    # Plot the data with improved styling
    ax.plot(formatted_dates, heat_max, color='black', label='Heat Max', linestyle ='-', marker='x')
    ax.plot(formatted_dates, heat_upper_quartile, color='orange', label='Heat 75th Percentile', linestyle ='-', marker='^')
    ax.plot(formatted_dates, heat_median, color='purple', label='Heat Median', linestyle ='-', marker='D')
    ax.plot(formatted_dates, heat_mean, color='red', label='Heat Mean', linestyle ='-', marker='s')
    ax.plot(formatted_dates, heat_lower_quartile, color='green', label='Heat 25th Percentile', linestyle ='-', marker='v')
    ax.plot(formatted_dates, heat_min, color='blue', label='Heat Min', linestyle ='-', marker='o')    
    
    #ax.plot(formatted_dates, tas_max, color='grey', label='Tas Max', linestyle='--', marker='x')
    #ax.plot(formatted_dates, tas_upper_quartile, color='brown', label='Tas 75th Percentile', linestyle='--', marker='^')
    #ax.plot(formatted_dates, tas_median, color='yellow', label='Tas Median', linestyle='--', marker='D')
    #ax.plot(formatted_dates, tas_mean, color='magenta', label='Tas Mean', linestyle='--', marker='s')
    #ax.plot(formatted_dates, tas_lower_quartile, color='lime', label='Tas 25th Percentile', linestyle='--', marker='v')
    #ax.plot(formatted_dates, tas_min, color='cyan', label='Tas Min', linestyle='--', marker='o')

    # Customize the plot
    ax.set_xlabel('Time')
    ax.set_ylabel('Mean Value (K)')
    ax.set_title("Yearly Mean Values from Lu and Romps' (2022) Heat Index")
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax.grid(True)
    plt.xticks(rotation=45, ha='right')
    
    # Reduces the number of x-axis ticks
    ax.xaxis.set_major_locator(mdates.YearLocator(10))  # Shows ticks and labels every 20 years
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y')) # Shows tick labels only as years. Change this to '%Y-%m' if you want to see the month as well but will likely look overcrowded with many years of data input
    plt.setp(ax.get_xticklabels(), ha='center')

    # Save and show the plot
    plt.tight_layout()
    plt.savefig(f'{output_folder}/yearly_temporal_mean_values_from_Lu_and_Romps.png')
    plt.show()

In [ ]:
def find_maximum_values_per_cell(heat_index_cube):
    heat_index_cube = heat_index_cube.copy()
    heat_index_cube.aggregated_by('season_year', iris.analysis.MAX)
    heat_index_cube.collapsed('time', iris.analysis.MAX)
    return heat_index_cube

def find_maximum_values_per_cell_for_tas(tas_cube):
    tas_cube_maximums = tas_cube.aggregated_by('season_year', iris.analysis.MAX)
    tas_cube_maximums = tas_cube_maximums.collapsed('time', iris.analysis.MAX)
    return tas_cube_maximums

def find_temporal_average_maximum_values_per_cell():

    heat_index_cube.aggregated_by('season_year', iris.analysis.MAX)
    heat_index_cube.collapsed('time', iris.analysis.MEAN)
    return heat_index_cube

In [ ]:
def calculate_statistics_per_decade(cube):
    decades = range(1951, 1994, 10)
    statistics = {}

    for start_year in decades:
        end_year = start_year + 9
        decade_cube = cube.extract(iris.Constraint(time=lambda cell: start_year <= cell.point.year <= end_year))
        
        min_val = decade_cube.collapsed('time', iris.analysis.MIN)
        # lower_quartile = decade_cube.collapsed('time', iris.analysis.PERCENTILE, percent=25)
        mean_val = decade_cube.collapsed('time', iris.analysis.MEAN)
        # median_val = decade_cube.collapsed('time', iris.analysis.MEDIAN)
        # upper_quartile = decade_cube.collapsed('time', iris.analysis.PERCENTILE, percent=75)
        max_val = decade_cube.collapsed('time', iris.analysis.MAX)
        
        statistics[start_year] = {
            'min': min_val,
            #'lower_quartile': lower_quartile,
            'mean': mean_val,
            #'median': median_val,
            #'upper_quartile': upper_quartile,
            'max': max_val
        }
    
    return statistics

def plot_decadal_maps(statistics, output_folder):
    metrics = ['min', 
               #'lower_quartile',
               'mean', 
               #'median', 
               #'upper_quartile', 
               'max']
    decades = sorted(statistics.keys())
    
    # Adjust ncols to exclude the final column (otherwise includes a decade after 1990)
    fig, axes = plt.subplots(nrows=len(metrics), ncols=len(decades) - 1, figsize=(18, 10), subplot_kw={'projection': ccrs.PlateCarree()})
    
    for i, metric in enumerate(metrics):
        for j, start_year in enumerate(decades[:-1]):  # Exclude the last decade
            ax = axes[i, j]
            if i == 0:  # Needed to change the titles of the subplots to be just at the top of the first row
                ax.set_title(f'{start_year} - {start_year + 9}')
            ax.set_ylabel(metric.capitalize())
            
            cube = statistics[start_year][metric]
            mesh = ax.pcolormesh(cube.coord('longitude').points, cube.coord('latitude').points, cube.data, cmap='Reds')
            fig.colorbar(mesh, ax=ax, orientation='horizontal', pad=0.02)
            shapefile.plot(ax=ax, edgecolor='black', facecolor='none')

    # Add row labels using fig.text for better control
    row_labels = metrics
    for row, label in enumerate(row_labels):
        fig.text(0.05, 0.79 - row * 0.31, label.capitalize(), va='center', ha='center', rotation='vertical', fontsize=14)
    
    plt.tight_layout(rect=[0.05, 0.01, 1, 0.95])  # Adjust the left margin to make space for the labels, otherwise overlaps
    plt.savefig(f'{output_folder}/decadal_heat_index_maps.png')
    plt.show()


In [ ]:
def plot_map_for_lu_and_romps_average_maximum_cell_values():
    cube = lu_and_romps_average_maximum_cell_values

    # Plotting
    fig, ax = plt.subplots(figsize=(10, 6))  # Adjust figsize as needed
    fig.suptitle('Heat Index Map: Spatial Distribution of Mean Values for Max Season Year')

    longitude = cube.coord('longitude').points
    latitude = cube.coord('latitude').points
    lon, lat = np.meshgrid(longitude, latitude)

    data_min = cube.data.min()
    data_max = cube.data.max()
    
    # Determine the appropriate colormap and normalization
    if data_max <= 0:
        cmap = 'Blues'
        norm = mcolors.Normalize(vmin=data_min, vmax=0)
        ticks = np.linspace(data_min, 0, 5)
    elif data_min > 0:
        cmap = 'Reds'
        norm = mcolors.Normalize(vmin=data_min, vmax=data_max)
        ticks = np.linspace(data_min, data_max, 5)
    else: # If the data crosses zero 
        vmin = min(data_min, 0)
        vmax = max(data_max, 0)
        vcenter = (vmin + vmax) / 2
        cmap = 'bwr'
        norm = TwoSlopeNorm(vmin=vmin, vcenter=vcenter, vmax=vmax)
        ticks = np.linspace(data_min, data_max, 5)
    
    mesh = ax.pcolormesh(lon, lat, cube.data, cmap=cmap, norm=norm, shading='auto')
    cbar = fig.colorbar(mesh, ax=ax, orientation='horizontal')
    
    # Set colorbar ticks and labels
    cbar.set_ticks(ticks)
    cbar.set_ticklabels([f'{val:.2f}' for val in ticks])

    ax.set_title('Lu and Romps Average Maximum Cell Values')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')

    shapefile.plot(ax=ax, color='none', edgecolor='black', linewidth=1)

    output_folder = f"outputs/heat_index_outputs"
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)
    
    plt.savefig(f'{output_folder}/lu_and_romps_average_maximum_cell_values_map.png')
    plt.show()

# Extra unused

In [ ]:
def temporally_plot_monthly_max_values_from_Lu_and_Romps_against_air_temperature():
    fig, ax = plt.subplots(figsize=(12, 8))

    # Calculate min, lower quartile, mean, median, upper quartile, and max for the heat_index_cube and tas_cube
    heat_min, heat_lower_quartile, heat_mean, heat_median, heat_upper_quartile, heat_max = calculate_statistics_max_yearly(heat_index_cube)
    tas_min, tas_lower_quartile, tas_mean, tas_median, tas_upper_quartile, tas_max = calculate_statistics_max_yearly(tas_cube)

    # Assuming `aggregated_cube` is already defined
    time_points = aggregated_cube.coord('time').points
    time_unit = aggregated_cube.coord('time').units

    # Convert numerical time points to human-readable dates
    readable_dates = cf_units.num2date(time_points, time_unit.origin, time_unit.calendar)
    formatted_dates = [date.strftime('%Y-%m') for date in readable_dates]

    # Plot the data with improved styling
    ax.plot(formatted_dates, heat_min, color='blue', label='HI Min', marker='o')
    ax.plot(formatted_dates, heat_lower_quartile, color='green', label='HI 25th Percentile', marker='v')
    ax.plot(formatted_dates, heat_mean, color='red', label='HI Mean', marker='s')
    ax.plot(formatted_dates, heat_median, color='purple', label='HI Median', marker='D')
    ax.plot(formatted_dates, heat_upper_quartile, color='orange', label='HI 75th Percentile', marker='^')
    ax.plot(formatted_dates, heat_max, color='black', label='HI Max', marker='x')

    ax.plot(formatted_dates, tas_min, color='cyan', label='AT Min', linestyle='--', marker='o')
    ax.plot(formatted_dates, tas_lower_quartile, color='lime', label='AT 25th Percentile', linestyle='--', marker='v')
    ax.plot(formatted_dates, tas_mean, color='magenta', label='AT Mean', linestyle='--', marker='s')
    ax.plot(formatted_dates, tas_median, color='yellow', label='AT Median', linestyle='--', marker='D')
    ax.plot(formatted_dates, tas_upper_quartile, color='brown', label='AT 75th Percentile', linestyle='--', marker='^')
    ax.plot(formatted_dates, tas_max, color='grey', label='AT Max', linestyle='--', marker='x')

    # Customize the plot
    ax.set_xlabel('Time')
    ax.set_ylabel('Max Value (K)')
    ax.set_title("Yearly Max Values from Lu and Romps' (2022) Heat Index and Air Temperature")
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax.grid(True)
    plt.xticks(rotation=90)

    # Reduces the number of x-axis ticks
    ax.xaxis.set_major_locator(mdates.YearLocator(20))  # Shows ticks and labels every 20 years
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y')) # Shows tick labels only as years. Change this to '%Y-%m' if you want to see the month as well but will likely look overcrowded with many years of data input
    plt.setp(ax.get_xticklabels(), ha='center')
    
    # Save and show the plot
    plt.tight_layout()
    plt.savefig(f'{output_folder}/yearly_temporal_max_values_from_Lu_and_Romps_against_air_temperature.png')
    plt.show()

In [ ]:
def plot_difference_between_lu_and_romps_and_air_temperature():
    fig, ax = plt.subplots(figsize=(12, 8))

    # Extract years and months
    years = np.unique(heat_index_cube.coord('season_year').points)
    months = np.unique(heat_index_cube.coord('month').points)

    # Calculate statistics for Lu and Romps heat index and air temperature
    heat_min, heat_lower_quartile, heat_mean, heat_median, heat_upper_quartile, heat_max = calculate_statistics_max_yearly(heat_index_cube)
    tas_min, tas_lower_quartile, tas_mean, tas_median, tas_upper_quartile, tas_max = calculate_statistics_max_yearly(tas_cube)

    # Assuming `aggregated_cube` is already defined
    time_points = aggregated_cube.coord('time').points
    time_unit = aggregated_cube.coord('time').units

    # Convert numerical time points to human-readable dates
    readable_dates = cf_units.num2date(time_points, time_unit.origin, time_unit.calendar)
    formatted_dates = [date.strftime('%Y-%m') for date in readable_dates]

    # Calculate differences
    diff_min = heat_min - tas_min
    diff_lower_quartile = heat_lower_quartile - tas_lower_quartile
    diff_mean = heat_mean - tas_mean
    diff_median = heat_median - tas_median
    diff_upper_quartile = heat_upper_quartile - tas_upper_quartile
    diff_max = heat_max - tas_max

    # Plot the differences
    ax.plot(formatted_dates, diff_min, color='blue', label='Min Difference', marker='o')
    ax.plot(formatted_dates, diff_lower_quartile, color='green', label='25th Percentile Difference', marker='v')
    ax.plot(formatted_dates, diff_mean, color='red', label='Mean Difference', marker='s')
    ax.plot(formatted_dates, diff_median, color='purple', label='Median Difference', marker='D')
    ax.plot(formatted_dates, diff_upper_quartile, color='orange', label='75th Percentile Difference', marker='^')
    ax.plot(formatted_dates, diff_max, color='black', label='Max Difference', marker='x')

    # Customize the plot
    ax.set_xlabel('Time')
    ax.set_ylabel('Temperature Difference (K)')
    ax.set_title("Differences Between Yearly Max Values from Lu and Romps' (2022) Heat Index and Air Temperature")
    ax.legend(loc='upper right')
    ax.grid(True)
    plt.xticks(rotation=45, ha='right')
    plt.xlim(formatted_dates[0], formatted_dates[-1])

    # Reduces the number of x-axis ticks
    ax.xaxis.set_major_locator(mdates.YearLocator(20))  # Shows ticks and labels every 20 years
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y')) # Shows tick labels only as years. Change this to '%Y-%m' if you want to see the month as well but will likely look overcrowded with many years of data input
    plt.setp(ax.get_xticklabels(), ha='center')
    
    # Save and show the plot
    plt.tight_layout()
    plt.savefig(f'{output_folder}/differences_between_yearly_temporal_max_values_from_Lu_and_Romps_against_air_temperature.png')
    plt.show()